# SHARE OF BIOGAS IN THE GAS SYSTEM

## Goal

This is an exercise in understanding an SQL request and translating it to pandas script. The goal is to generate data for the graph on https://www.energidataservice.dk/. 

## Source SQL

```SQL=
with c1 as (
	select cast(date_trunc('month', "GasDay") as date) as mn,  Sum("KWhFromBiogas") as bio, Sum(ABS("KWhToDenmark")) as cons
	from "gasflow" 
	where cast(date_trunc('month', now() AT TIME ZONE 'UTC' - interval '2 year') as date) <= "GasDay" AND "GasDay" < cast(date_trunc('month', now() AT TIME ZONE 'UTC') as date)
	group by cast(date_trunc('month', "GasDay") as date)
)
, c2 as (
	select mn, cons, sum(cons) OVER (order by mn rows between 11 preceding and current row) as yearcons, bio,
	date_part('days', date_trunc('month', mn) - '1DAY - 1MONTH'::INTERVAL) as days
	from c1
)
select LEFT(TO_CHAR(mn, 'MON'), 2) as "GasDay", round( CAST(float8 (bio * 365 / days / yearcons * 100) as numeric), 1) as "KWhFromBiogas"
from c2 
where mn >= cast(date_trunc('month', now() AT TIME ZONE 'Europe/Rome' - interval '1 year') as date)
order by mn
```

## Source Data

The source data was downloaded for this exercise from https://www.energidataservice.dk/dataset/9c12a3dd-71e8-433b-b78e-72021838d987/resource_extract/f40147ba-8e83-438e-9e89-bfddd0cba81f with filters for the GasDay for the last 2 years.

In [1]:
import pandas as pd

By downloading the data for the last 2 years, the following filter was already applied:
```SQL
where cast(date_trunc('month', now() AT TIME ZONE 'UTC' - interval '2 year') as date) <= "GasDay" AND "GasDay" < cast(date_trunc('month', now() AT TIME ZONE 'UTC') as date)
```

In [5]:
kwh_last_2_years = pd.read_csv('commercial_gas_amounts_data_last_2_years.csv', index_col='GasDay', parse_dates=True)
kwh_last_2_years.head()

,KWhFromBiogas,KWhToDenmark,KWhFromNorthSea,KWhToOrFromStorage,KWhToOrFromGermany,KWhToSweden
GasDay,,,,,,
2019-11-12,8707816,-110556673,3960000,-3940871,120516272,-25526524
2019-11-11,8716894,-107407541,3840000,-5146203,126948272,-29393329
2019-11-10,9110262,-95785479,3960000,-23620807,127158252,-25523940
2019-11-09,9258907,-100999820,3615000,-14062004,138233272,-24293794
2019-11-08,9171005,-112648950,3360000,-9213003,141357712,-25072131


In [17]:
"""
with c1 as (
	select cast(date_trunc('month', "GasDay") as date) as mn,  Sum("KWhFromBiogas") as bio, Sum(ABS("KWhToDenmark")) as cons
	from "gasflow" 
	where cast(date_trunc('month', now() AT TIME ZONE 'UTC' - interval '2 year') as date) <= "GasDay" AND "GasDay" < cast(date_trunc('month', now() AT TIME ZONE 'UTC') as date)
	group by cast(date_trunc('month', "GasDay") as date)
)
"""

kwh_by_month = kwh_last_2_years.abs().resample('M').sum()
c1 = pd.DataFrame(index=kwh_by_month.index)
c1['mn'] = kwh_by_month.index
c1['bio'] = kwh_by_month.KWhFromBiogas
c1['cons'] = kwh_by_month.KWhToDenmark
c1.head()

,mn,bio,cons
GasDay,,,
2017-11-30,2017-11-30,93042317,1980326988
2017-12-31,2017-12-31,166915253,3402667812
2018-01-31,2018-01-31,162328416,4156028550
2018-02-28,2018-02-28,149595805,4290611682
2018-03-31,2018-03-31,162545142,4311347919


In [26]:
"""
, c2 as (
	select mn, cons, sum(cons) OVER (order by mn rows between 11 preceding and current row) as yearcons, bio,
	date_part('days', date_trunc('month', mn) - '1DAY - 1MONTH'::INTERVAL) as days
	from c1 
)
"""
c2 = c1.copy(deep=True)
c2['yearcons'] = c2['cons'].rolling(window=12, min_periods=12).sum()
c2['days'] = c2.index.daysinmonth
c2

,mn,bio,cons,yearcons,days
GasDay,,,,,
2017-11-30,2017-11-30,93042317,1980326988,NaN,30
2017-12-31,2017-12-31,166915253,3402667812,NaN,31
2018-01-31,2018-01-31,162328416,4156028550,NaN,31
2018-02-28,2018-02-28,149595805,4290611682,NaN,28
2018-03-31,2018-03-31,162545142,4311347919,NaN,31
2018-04-30,2018-04-30,157072701,2192381331,NaN,30
2018-05-31,2018-05-31,171497795,1425508440,NaN,31
2018-06-30,2018-06-30,166200918,1158314496,NaN,30
2018-07-31,2018-07-31,185848469,1007209600,NaN,31


In [25]:
# test that all this rolling business is indeed what we want
test_12_months = c2[c2['mn'] <= '2018-10-31']
test_12_months.cons.sum()

29045217891

In [27]:
"""
select LEFT(TO_CHAR(mn, 'MON'), 2) as "GasDay", round( CAST(float8 (bio * 365 / days / yearcons * 100) as numeric), 1) as "KWhFromBiogas"
from c2 
where mn >= cast(date_trunc('month', now() AT TIME ZONE 'Europe/Rome' - interval '1 year') as date)
order by mn
"""

c2['KWhFromBiogas'] = (c2.bio * 365 / c2.days / c2.yearcons * 100)
c2

,mn,bio,cons,yearcons,days,KWhFromBiogas
GasDay,,,,,,
2017-11-30,2017-11-30,93042317,1980326988,NaN,30,NaN
2017-12-31,2017-12-31,166915253,3402667812,NaN,31,NaN
2018-01-31,2018-01-31,162328416,4156028550,NaN,31,NaN
2018-02-28,2018-02-28,149595805,4290611682,NaN,28,NaN
2018-03-31,2018-03-31,162545142,4311347919,NaN,31,NaN
2018-04-30,2018-04-30,157072701,2192381331,NaN,30,NaN
2018-05-31,2018-05-31,171497795,1425508440,NaN,31,NaN
2018-06-30,2018-06-30,166200918,1158314496,NaN,30,NaN
2018-07-31,2018-07-31,185848469,1007209600,NaN,31,NaN


And we've got the same numbers as in the graph!